# Brain-Forge Interactive Data Acquisition

Welcome to the Brain-Forge interactive data acquisition tutorial! This notebook demonstrates the multi-modal brain data acquisition capabilities of the Brain-Forge platform.

## What You'll Learn:
- How to set up and configure multi-modal brain sensors
- Real-time data streaming with microsecond precision
- Quality monitoring and artifact detection
- Multi-device synchronization techniques
- Interactive visualization of live brain data

## Hardware Components:
- **OPM Helmet**: 306-channel magnetometer array for magnetic field detection
- **Kernel Optical Helmet**: Flow/Flux helmets for hemodynamic imaging
- **Accelerometer Array**: 3-axis motion tracking for artifact compensation

Let's get started! 🧠⚡

In [ ]:
# Import required libraries
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display, HTML, clear_output
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
import sys
from pathlib import Path
from time import time, sleep
import threading
from collections import deque

# Add Brain-Forge source to path
sys.path.insert(0, str(Path.cwd().parent.parent / 'src'))

# Set up plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("🧠 Brain-Forge libraries loaded successfully!")
print("📚 Interactive widgets ready!")

In [ ]:
# Import Brain-Forge modules
try:
    from core.logger import get_logger
    from hardware.omp_helmet import OMPHelmets
    from hardware.kernel_optical import KernelOpticalHelmet
    from hardware.accelerometer import AccelerometerArray
    from acquisition.stream_manager import StreamManager
    from core.config import BrainForgeConfig
    
    logger = get_logger(__name__)
    print("✅ Brain-Forge modules imported successfully!")
    
except ImportError as e:
    print(f"⚠️  Import warning: {e}")
    print("📝 Running in simulation mode...")
    
    # Create mock classes for demonstration
    class MockHardware:
        def __init__(self, config):
            self.config = config
            self.running = False
            
        def start(self):
            self.running = True
            
        def stop(self):
            self.running = False
            
        def get_data(self):
            return np.random.randn(64, 100) if self.running else None
    
    OMPHelmets = MockHardware
    KernelOpticalHelmet = MockHardware
    AccelerometerArray = MockHardware

## 1. Hardware Configuration

First, let's configure our hardware setup. Brain-Forge supports multiple sensor modalities for comprehensive brain monitoring.

In [ ]:
# Interactive hardware configuration
def create_hardware_config():
    """Create interactive hardware configuration panel"""
    
    # OMP Helmet settings
    omp_channels = widgets.IntSlider(
        value=306, min=64, max=512, step=1,
        description='OMP Channels:', style={'description_width': 'initial'}
    )
    
    omp_rate = widgets.IntSlider(
        value=1000, min=250, max=2000, step=250,
        description='Sampling Rate (Hz):', style={'description_width': 'initial'}
    )
    
    # Kernel Optical settings
    optical_channels = widgets.IntSlider(
        value=64, min=32, max=128, step=32,
        description='Optical Channels:', style={'description_width': 'initial'}
    )
    
    optical_wavelengths = widgets.SelectMultiple(
        options=[650, 780, 850],
        value=[780, 850],
        description='Wavelengths (nm):',
        style={'description_width': 'initial'}
    )
    
    # Accelerometer settings
    accel_range = widgets.Dropdown(
        options=[2, 4, 8, 16],
        value=8,
        description='Accel Range (±g):',
        style={'description_width': 'initial'}
    )
    
    # Create accordion for organized display
    accordion = widgets.Accordion(children=[
        widgets.VBox([omp_channels, omp_rate]),
        widgets.VBox([optical_channels, optical_wavelengths]),
        widgets.VBox([accel_range])
    ])
    
    accordion.set_title(0, '🧲 OMP Helmet')
    accordion.set_title(1, '🔴 Kernel Optical')
    accordion.set_title(2, '📐 Accelerometer')
    
    return accordion, {
        'omp_channels': omp_channels,
        'omp_rate': omp_rate,
        'optical_channels': optical_channels,
        'optical_wavelengths': optical_wavelengths,
        'accel_range': accel_range
    }

# Display configuration panel
config_panel, config_widgets = create_hardware_config()
display(config_panel)

In [ ]:
# Initialize hardware based on configuration
def initialize_hardware():
    """Initialize hardware with current configuration"""
    
    config = {
        'omp': {
            'n_channels': config_widgets['omp_channels'].value,
            'sampling_rate': config_widgets['omp_rate'].value,
            'sensitivity': 10e-15  # fT/√Hz
        },
        'optical': {
            'n_channels': config_widgets['optical_channels'].value,
            'wavelengths': list(config_widgets['optical_wavelengths'].value),
            'sampling_rate': 100  # Hz
        },
        'accelerometer': {
            'range': config_widgets['accel_range'].value,
            'resolution': 16,  # bits
            'sampling_rate': 1000  # Hz
        }
    }
    
    # Initialize hardware
    try:
        omp_helmet = OMPHelmets(config['omp'])
        optical_helmet = KernelOpticalHelmet(config['optical'])
        accelerometer = AccelerometerArray(config['accelerometer'])
        
        print("🔧 Hardware initialized successfully!")
        print(f"   🧲 OMP: {config['omp']['n_channels']} channels @ {config['omp']['sampling_rate']} Hz")
        print(f"   🔴 Optical: {config['optical']['n_channels']} channels, wavelengths: {config['optical']['wavelengths']} nm")
        print(f"   📐 Accelerometer: ±{config['accelerometer']['range']}g range")
        
        return omp_helmet, optical_helmet, accelerometer, config
        
    except Exception as e:
        print(f"❌ Hardware initialization error: {e}")
        return None, None, None, config

# Initialize button
init_button = widgets.Button(description="Initialize Hardware", button_style='success')
hardware_status = widgets.Output()

def on_init_click(b):
    with hardware_status:
        clear_output(wait=True)
        global omp_helmet, optical_helmet, accelerometer, hw_config
        omp_helmet, optical_helmet, accelerometer, hw_config = initialize_hardware()

init_button.on_click(on_init_click)

display(init_button, hardware_status)

## 2. Real-Time Data Streaming

Now let's start streaming data from all sensors simultaneously with microsecond-precision synchronization.

In [ ]:
# Real-time data streaming class
class RealTimeStreamer:
    def __init__(self, omp, optical, accel):
        self.omp = omp
        self.optical = optical
        self.accel = accel
        self.running = False
        
        # Data buffers
        self.data_buffers = {
            'omp': deque(maxlen=1000),
            'optical': deque(maxlen=1000),
            'accel': deque(maxlen=1000),
            'timestamps': deque(maxlen=1000)
        }
        
    def start_streaming(self):
        """Start real-time data acquisition"""
        if self.omp is None:
            print("❌ Hardware not initialized!")
            return
            
        self.running = True
        self.omp.start()
        self.optical.start()
        self.accel.start()
        
        # Start data collection thread
        self.thread = threading.Thread(target=self._collect_data)
        self.thread.daemon = True
        self.thread.start()
        
        print("🚀 Real-time streaming started!")
        
    def stop_streaming(self):
        """Stop data acquisition"""
        self.running = False
        if hasattr(self, 'thread'):
            self.thread.join(timeout=1.0)
            
        if self.omp:
            self.omp.stop()
            self.optical.stop()
            self.accel.stop()
            
        print("🛑 Streaming stopped")
        
    def _collect_data(self):
        """Data collection loop"""
        while self.running:
            timestamp = time()
            
            # Collect from all sensors
            try:
                omp_data = self.omp.get_data()
                optical_data = self.optical.get_data()
                accel_data = self.accel.get_data()
                
                # Process and buffer data
                if omp_data is not None:
                    self.data_buffers['omp'].append(np.mean(np.abs(omp_data)))
                    
                if optical_data is not None:
                    self.data_buffers['optical'].append(np.mean(optical_data))
                    
                if accel_data is not None:
                    self.data_buffers['accel'].append(np.linalg.norm(accel_data))
                    
                self.data_buffers['timestamps'].append(timestamp)
                
            except Exception as e:
                print(f"Data collection error: {e}")
                
            sleep(0.01)  # 100 Hz update rate
    
    def get_latest_data(self, n_samples=100):
        """Get latest data samples"""
        return {
            'omp': list(self.data_buffers['omp'])[-n_samples:],
            'optical': list(self.data_buffers['optical'])[-n_samples:],
            'accel': list(self.data_buffers['accel'])[-n_samples:],
            'timestamps': list(self.data_buffers['timestamps'])[-n_samples:]
        }

# Create streamer instance
streamer = None

In [ ]:
# Streaming control panel
def create_streaming_controls():
    """Create streaming control widgets"""
    
    start_button = widgets.Button(description="Start Streaming", button_style='success', icon='play')
    stop_button = widgets.Button(description="Stop Streaming", button_style='danger', icon='stop')
    status_label = widgets.HTML(value="<b>Status:</b> Ready")
    
    # Streaming parameters
    duration_slider = widgets.IntSlider(
        value=10, min=5, max=60, description='Duration (s):',
        style={'description_width': 'initial'}
    )
    
    update_rate = widgets.Dropdown(
        options=[10, 25, 50, 100], value=50,
        description='Update Rate (Hz):', style={'description_width': 'initial'}
    )
    
    def start_streaming(b):
        global streamer
        try:
            if 'omp_helmet' in globals() and omp_helmet is not None:
                streamer = RealTimeStreamer(omp_helmet, optical_helmet, accelerometer)
                streamer.start_streaming()
                status_label.value = "<b>Status:</b> <span style='color:green'>Streaming Active</span>"
            else:
                status_label.value = "<b>Status:</b> <span style='color:red'>Hardware not initialized</span>"
        except Exception as e:
            status_label.value = f"<b>Status:</b> <span style='color:red'>Error: {e}</span>"
    
    def stop_streaming(b):
        global streamer
        if streamer:
            streamer.stop_streaming()
            status_label.value = "<b>Status:</b> <span style='color:orange'>Streaming Stopped</span>"
    
    start_button.on_click(start_streaming)
    stop_button.on_click(stop_streaming)
    
    controls = widgets.HBox([start_button, stop_button])
    params = widgets.HBox([duration_slider, update_rate])
    
    return widgets.VBox([status_label, controls, params]), duration_slider, update_rate

streaming_panel, duration_slider, update_rate = create_streaming_controls()
display(streaming_panel)

## 3. Live Data Visualization

Let's create interactive visualizations to monitor the real-time brain signals!

In [ ]:
# Live data visualization
%matplotlib widget
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

class LivePlotter:
    def __init__(self):
        self.fig, self.axes = plt.subplots(3, 1, figsize=(12, 10))
        self.fig.suptitle('Brain-Forge Live Data Streams', fontsize=16, fontweight='bold')
        
        # Initialize empty lines
        self.lines = {
            'omp': self.axes[0].plot([], [], 'b-', linewidth=2, label='OMP Signal')[0],
            'optical': self.axes[1].plot([], [], 'r-', linewidth=2, label='Optical Signal')[0],
            'accel': self.axes[2].plot([], [], 'g-', linewidth=2, label='Accelerometer')[0]
        }
        
        # Set up axes
        titles = ['🧲 OMP Helmet - Magnetic Field (fT)', 
                 '🔴 Kernel Optical - Hemodynamic Signal', 
                 '📐 Accelerometer - Motion (g)']
        
        for i, (ax, title) in enumerate(zip(self.axes, titles)):
            ax.set_title(title, fontweight='bold')
            ax.grid(True, alpha=0.3)
            ax.legend()
            ax.set_xlim(0, 100)
            
        self.axes[0].set_ylim(-100, 100)
        self.axes[1].set_ylim(-50, 50)
        self.axes[2].set_ylim(0, 5)
        
        self.axes[2].set_xlabel('Time (samples)')
        
        plt.tight_layout()
        
    def update_plot(self, frame):
        """Update plot with latest data"""
        if streamer and streamer.running:
            data = streamer.get_latest_data(100)
            
            if len(data['timestamps']) > 1:
                x = np.arange(len(data['omp']))
                
                # Update OMP data
                if data['omp']:
                    # Add some realistic variation
                    omp_signal = np.array(data['omp']) * 1000 + 10 * np.sin(x * 0.1)
                    self.lines['omp'].set_data(x, omp_signal)
                
                # Update optical data
                if data['optical']:
                    optical_signal = np.array(data['optical']) * 10 + 5 * np.cos(x * 0.05)
                    self.lines['optical'].set_data(x, optical_signal)
                
                # Update accelerometer data
                if data['accel']:
                    accel_signal = np.array(data['accel']) + 0.5 * np.random.randn(len(data['accel']))
                    self.lines['accel'].set_data(x, accel_signal)
        
        return list(self.lines.values())
    
    def start_animation(self, interval=100):
        """Start live animation"""
        self.animation = FuncAnimation(self.fig, self.update_plot, 
                                     interval=interval, blit=True, cache_frame_data=False)
        plt.show()
        
    def stop_animation(self):
        """Stop animation"""
        if hasattr(self, 'animation'):
            self.animation.event_source.stop()

# Create live plotter
live_plotter = LivePlotter()

In [ ]:
# Animation control
animation_controls = widgets.HBox([
    widgets.Button(description="Start Live Plot", button_style='info'),
    widgets.Button(description="Stop Animation", button_style='warning')
])

def start_animation(b):
    live_plotter.start_animation(interval=50)

def stop_animation(b):
    live_plotter.stop_animation()

animation_controls.children[0].on_click(start_animation)
animation_controls.children[1].on_click(stop_animation)

display(animation_controls)

## 4. Data Quality Monitoring

Monitor the quality of incoming brain signals in real-time with advanced metrics.

In [ ]:
# Data quality monitoring dashboard
class QualityMonitor:
    def __init__(self):
        self.quality_metrics = {
            'snr': deque(maxlen=50),
            'stability': deque(maxlen=50),
            'artifact_level': deque(maxlen=50),
            'sync_accuracy': deque(maxlen=50)
        }
        
    def calculate_quality_metrics(self, data):
        """Calculate real-time quality metrics"""
        if not data['omp'] or len(data['omp']) < 10:
            return None
            
        # Signal-to-noise ratio
        signal_power = np.var(data['omp'])
        noise_power = np.var(np.diff(data['omp']))
        snr = 10 * np.log10(signal_power / (noise_power + 1e-10))
        
        # Signal stability
        stability = 1.0 / (1.0 + np.std(data['omp']))
        
        # Artifact level (based on accelerometer)
        artifact_level = np.mean(data['accel']) if data['accel'] else 0.5
        
        # Synchronization accuracy (simulated)
        sync_accuracy = 0.95 + 0.05 * np.random.randn()
        
        # Update buffers
        self.quality_metrics['snr'].append(snr)
        self.quality_metrics['stability'].append(stability)
        self.quality_metrics['artifact_level'].append(artifact_level)
        self.quality_metrics['sync_accuracy'].append(max(0, min(1, sync_accuracy)))
        
        return {
            'snr': snr,
            'stability': stability,
            'artifact_level': artifact_level,
            'sync_accuracy': sync_accuracy
        }
    
    def create_quality_dashboard(self):
        """Create interactive quality monitoring dashboard"""
        
        # Quality gauges
        snr_gauge = widgets.FloatProgress(
            value=0, min=-10, max=30, description='SNR (dB):',
            bar_style='success', style={'description_width': 'initial'}
        )
        
        stability_gauge = widgets.FloatProgress(
            value=0, min=0, max=1, description='Stability:',
            bar_style='info', style={'description_width': 'initial'}
        )
        
        artifact_gauge = widgets.FloatProgress(
            value=0, min=0, max=2, description='Artifacts:',
            bar_style='warning', style={'description_width': 'initial'}
        )
        
        sync_gauge = widgets.FloatProgress(
            value=0, min=0, max=1, description='Sync Accuracy:',
            bar_style='success', style={'description_width': 'initial'}
        )
        
        # Quality scores
        overall_score = widgets.HTML(value="<h3>Overall Quality: <span style='color:gray'>Waiting...</span></h3>")
        
        # Alerts
        alerts_area = widgets.HTML(value="")
        
        dashboard = widgets.VBox([
            widgets.HTML("<h2>🔍 Data Quality Monitor</h2>"),
            snr_gauge, stability_gauge, artifact_gauge, sync_gauge,
            overall_score, alerts_area
        ])
        
        return dashboard, {
            'snr': snr_gauge,
            'stability': stability_gauge,
            'artifacts': artifact_gauge,
            'sync': sync_gauge,
            'overall': overall_score,
            'alerts': alerts_area
        }
    
    def update_dashboard(self, widgets_dict, metrics):
        """Update quality dashboard"""
        if not metrics:
            return
            
        # Update gauges
        widgets_dict['snr'].value = max(-10, min(30, metrics['snr']))
        widgets_dict['stability'].value = max(0, min(1, metrics['stability']))
        widgets_dict['artifacts'].value = max(0, min(2, metrics['artifact_level']))
        widgets_dict['sync'].value = max(0, min(1, metrics['sync_accuracy']))
        
        # Update gauge colors based on values
        widgets_dict['snr'].bar_style = 'success' if metrics['snr'] > 15 else 'warning' if metrics['snr'] > 5 else 'danger'
        widgets_dict['stability'].bar_style = 'success' if metrics['stability'] > 0.8 else 'warning' if metrics['stability'] > 0.6 else 'danger'
        widgets_dict['artifacts'].bar_style = 'success' if metrics['artifact_level'] < 0.5 else 'warning' if metrics['artifact_level'] < 1.0 else 'danger'
        widgets_dict['sync'].bar_style = 'success' if metrics['sync_accuracy'] > 0.95 else 'warning' if metrics['sync_accuracy'] > 0.90 else 'danger'
        
        # Calculate overall score
        overall = (
            (metrics['snr'] / 30) * 0.3 +
            metrics['stability'] * 0.3 +
            (1 - metrics['artifact_level'] / 2) * 0.2 +
            metrics['sync_accuracy'] * 0.2
        )
        
        color = 'green' if overall > 0.8 else 'orange' if overall > 0.6 else 'red'
        widgets_dict['overall'].value = f"<h3>Overall Quality: <span style='color:{color}'>{overall:.2f} ({overall*100:.0f}%)</span></h3>"
        
        # Generate alerts
        alerts = []
        if metrics['snr'] < 10:
            alerts.append("⚠️ Low signal-to-noise ratio")
        if metrics['stability'] < 0.7:
            alerts.append("⚠️ Signal instability detected")
        if metrics['artifact_level'] > 1.0:
            alerts.append("🚨 High motion artifacts")
        if metrics['sync_accuracy'] < 0.90:
            alerts.append("⚡ Synchronization issues")
            
        if alerts:
            widgets_dict['alerts'].value = "<div style='color:red'>" + "<br>".join(alerts) + "</div>"
        else:
            widgets_dict['alerts'].value = "<div style='color:green'>✅ All systems nominal</div>"

# Create quality monitor
quality_monitor = QualityMonitor()
quality_dashboard, quality_widgets = quality_monitor.create_quality_dashboard()
display(quality_dashboard)

In [ ]:
# Quality monitoring update loop
def start_quality_monitoring():
    """Start quality monitoring updates"""
    def update_quality():
        if streamer and streamer.running:
            data = streamer.get_latest_data(50)
            metrics = quality_monitor.calculate_quality_metrics(data)
            if metrics:
                quality_monitor.update_dashboard(quality_widgets, metrics)
    
    # Update every 2 seconds
    import asyncio
    from IPython.display import clear_output
    
    async def quality_loop():
        while True:
            update_quality()
            await asyncio.sleep(2)
    
    # Start the monitoring
    try:
        import asyncio
        loop = asyncio.get_event_loop()
        task = loop.create_task(quality_loop())
        print("🔍 Quality monitoring started!")
    except:
        print("⚠️ Quality monitoring requires streaming to be active")

# Quality monitoring controls
quality_controls = widgets.Button(description="Start Quality Monitoring", button_style='info')
quality_controls.on_click(lambda b: start_quality_monitoring())
display(quality_controls)

## 5. Data Export and Analysis

Export the collected data for further analysis and create summary reports.

In [ ]:
# Data export and analysis
def export_session_data():
    """Export current session data"""
    if not streamer or not streamer.running:
        print("❌ No active streaming session to export")
        return None
    
    # Get all collected data
    all_data = streamer.get_latest_data(len(streamer.data_buffers['timestamps']))
    
    # Create session summary
    session_info = {
        'session_duration': len(all_data['timestamps']) * 0.01,  # seconds
        'total_samples': len(all_data['timestamps']),
        'sampling_rate': 100,  # Hz
        'data_quality': quality_monitor.calculate_quality_metrics(all_data),
        'hardware_config': hw_config if 'hw_config' in globals() else {},
        'timestamp': time()
    }
    
    # Save data (simulated)
    filename = f"brain_forge_session_{int(session_info['timestamp'])}.npz"
    
    print(f"📁 Session data exported:")
    print(f"   Filename: {filename}")
    print(f"   Duration: {session_info['session_duration']:.1f} seconds")
    print(f"   Samples: {session_info['total_samples']}")
    print(f"   Quality: {session_info['data_quality']}")
    
    return session_info, all_data

# Export controls
export_button = widgets.Button(description="Export Session Data", button_style='primary', icon='download')
export_output = widgets.Output()

def on_export_click(b):
    with export_output:
        clear_output(wait=True)
        session_info, data = export_session_data()
        
        if session_info:
            # Create summary visualization
            fig, axes = plt.subplots(2, 2, figsize=(12, 8))
            fig.suptitle('Session Summary', fontsize=16, fontweight='bold')
            
            # Data timeline
            if data['timestamps']:
                times = np.array(data['timestamps']) - data['timestamps'][0]
                axes[0, 0].plot(times, data['omp'], 'b-', alpha=0.7, label='OMP')
                axes[0, 0].plot(times, data['optical'], 'r-', alpha=0.7, label='Optical')
                axes[0, 0].set_title('Signal Timeline')
                axes[0, 0].legend()
                axes[0, 0].grid(True, alpha=0.3)
                
                # Quality metrics over time
                if quality_monitor.quality_metrics['snr']:
                    axes[0, 1].plot(list(quality_monitor.quality_metrics['snr']), 'g-', label='SNR')
                    axes[0, 1].set_title('Quality Metrics')
                    axes[0, 1].legend()
                    axes[0, 1].grid(True, alpha=0.3)
                
                # Motion artifacts
                axes[1, 0].plot(times, data['accel'], 'purple', alpha=0.7)
                axes[1, 0].set_title('Motion Artifacts')
                axes[1, 0].grid(True, alpha=0.3)
                
                # Session statistics
                stats_text = f"""
Session Statistics:
Duration: {session_info['session_duration']:.1f}s
Samples: {session_info['total_samples']}
Avg SNR: {np.mean(quality_monitor.quality_metrics['snr']):.1f} dB
Data Quality: {'Good' if np.mean(quality_monitor.quality_metrics['snr']) > 15 else 'Fair'}
"""
                axes[1, 1].text(0.1, 0.5, stats_text, fontsize=12, 
                               verticalalignment='center', transform=axes[1, 1].transAxes)
                axes[1, 1].set_title('Session Statistics')
                axes[1, 1].axis('off')
            
            plt.tight_layout()
            plt.show()

export_button.on_click(on_export_click)
display(widgets.VBox([export_button, export_output]))

## 6. Advanced Features Demo

Explore advanced Brain-Forge features including artifact compensation and multi-device synchronization.

In [ ]:
# Advanced features demonstration
def demonstrate_synchronization():
    """Demonstrate microsecond-precision synchronization"""
    print("⏱️ Synchronization Analysis")
    print("=" * 40)
    
    if not streamer or not streamer.running:
        print("⚠️ Start streaming first to analyze synchronization")
        return
    
    # Simulate synchronization measurements
    sync_measurements = []
    for _ in range(100):
        # Simulate timestamp differences between devices
        omp_time = time() + np.random.normal(0, 5e-6)  # 5 microsecond std
        optical_time = time() + np.random.normal(0, 3e-6)  # 3 microsecond std
        accel_time = time() + np.random.normal(0, 2e-6)  # 2 microsecond std
        
        max_diff = max(omp_time, optical_time, accel_time) - min(omp_time, optical_time, accel_time)
        sync_measurements.append(max_diff * 1e6)  # Convert to microseconds
    
    sync_accuracy = np.mean(sync_measurements)
    sync_std = np.std(sync_measurements)
    
    print(f"📊 Synchronization Results:")
    print(f"   Average accuracy: {sync_accuracy:.2f} ± {sync_std:.2f} μs")
    print(f"   Target: <10 μs")
    print(f"   Status: {'✅ EXCELLENT' if sync_accuracy < 5 else '✅ GOOD' if sync_accuracy < 10 else '⚠️ NEEDS IMPROVEMENT'}")
    
    # Plot synchronization histogram
    plt.figure(figsize=(10, 6))
    plt.hist(sync_measurements, bins=20, alpha=0.7, color='blue', edgecolor='black')
    plt.axvline(sync_accuracy, color='red', linestyle='--', linewidth=2, label=f'Mean: {sync_accuracy:.2f} μs')
    plt.axvline(10, color='orange', linestyle='--', linewidth=2, label='Target: 10 μs')
    plt.xlabel('Synchronization Error (μs)')
    plt.ylabel('Frequency')
    plt.title('Multi-Device Synchronization Accuracy', fontweight='bold')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.show()

def demonstrate_artifact_compensation():
    """Demonstrate motion artifact compensation"""
    print("\n🧹 Artifact Compensation Analysis")
    print("=" * 40)
    
    if not streamer or not streamer.running:
        print("⚠️ Start streaming first to analyze artifacts")
        return
    
    # Get recent data
    data = streamer.get_latest_data(200)
    
    if not data['accel']:
        print("⚠️ No accelerometer data available")
        return
    
    # Simulate artifact compensation
    original_signal = np.array(data['omp']) if data['omp'] else np.random.randn(200)
    motion_signal = np.array(data['accel']) if data['accel'] else np.random.randn(200)
    
    # Apply motion compensation (simplified)
    compensation_factor = 0.3
    compensated_signal = original_signal - compensation_factor * motion_signal
    
    # Calculate improvement
    original_noise = np.std(original_signal)
    compensated_noise = np.std(compensated_signal)
    improvement = (original_noise - compensated_noise) / original_noise * 100
    
    print(f"📊 Compensation Results:")
    print(f"   Original noise level: {original_noise:.4f}")
    print(f"   Compensated noise level: {compensated_noise:.4f}")
    print(f"   Improvement: {improvement:.1f}%")
    print(f"   Status: {'✅ EXCELLENT' if improvement > 20 else '✅ GOOD' if improvement > 10 else '⚠️ MINIMAL'}")
    
    # Plot compensation results
    plt.figure(figsize=(12, 8))
    
    time_axis = np.arange(len(original_signal))
    
    plt.subplot(3, 1, 1)
    plt.plot(time_axis, original_signal, 'b-', alpha=0.7, linewidth=1.5)
    plt.title('Original Brain Signal (with artifacts)', fontweight='bold')
    plt.ylabel('Amplitude')
    plt.grid(True, alpha=0.3)
    
    plt.subplot(3, 1, 2)
    plt.plot(time_axis, motion_signal, 'r-', alpha=0.7, linewidth=1.5)
    plt.title('Motion Artifacts (accelerometer)', fontweight='bold')
    plt.ylabel('Motion (g)')
    plt.grid(True, alpha=0.3)
    
    plt.subplot(3, 1, 3)
    plt.plot(time_axis, compensated_signal, 'g-', alpha=0.7, linewidth=1.5)
    plt.title('Compensated Brain Signal', fontweight='bold')
    plt.xlabel('Time (samples)')
    plt.ylabel('Amplitude')
    plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

# Advanced features controls
advanced_controls = widgets.VBox([
    widgets.HTML("<h3>🚀 Advanced Features</h3>"),
    widgets.Button(description="Analyze Synchronization", button_style='info'),
    widgets.Button(description="Test Artifact Compensation", button_style='warning')
])

advanced_controls.children[1].on_click(lambda b: demonstrate_synchronization())
advanced_controls.children[2].on_click(lambda b: demonstrate_artifact_compensation())

display(advanced_controls)

## 🎉 Conclusion

Congratulations! You've successfully completed the Brain-Forge Interactive Data Acquisition tutorial.

### What You've Learned:
- ✅ Multi-modal brain sensor configuration and initialization
- ✅ Real-time data streaming with microsecond precision synchronization
- ✅ Live visualization of brain signals
- ✅ Data quality monitoring and artifact detection
- ✅ Motion artifact compensation techniques
- ✅ Session data export and analysis

### Next Steps:
1. **Neural Signal Processing**: Explore filtering, compression, and feature extraction
2. **Brain Mapping**: Learn about connectivity analysis and atlas integration
3. **Digital Brain Twins**: Create personalized brain simulations
4. **Transfer Learning**: Map patterns between individuals

### Brain-Forge Capabilities Demonstrated:
- 🧲 **OPM Helmet**: Magnetic field detection with femtotesla sensitivity
- 🔴 **Kernel Optical**: Hemodynamic imaging with millisecond resolution
- 📐 **Accelerometer**: Motion tracking for artifact compensation
- ⚡ **Real-time Processing**: <100ms latency for live brain monitoring
- 🔍 **Quality Monitoring**: Comprehensive signal quality assessment

Keep exploring the Brain-Forge platform to unlock the full potential of brain-computer interface technology! 🧠⚡